Note: this file relies on prep_audio_validation.ipynb having been executed previously.

In [1]:
import os
import polars as pl

In [11]:
df = pl.read_csv("../data/speech_validation_small.csv")
df.head()

file_id,speaker_est,start,duration,end,speaker_val
str,str,f64,f64,f64,str
"""gen_2000_debat…","""SPEAKER_17""",245.068,4.873,249.941,"""other"""
"""gen_2000_debat…","""SPEAKER_16""",250.552,25.908,276.46,"""gore"""
"""gen_2000_debat…","""SPEAKER_16""",277.852,11.053,288.905,"""gore"""
"""gen_2000_debat…","""SPEAKER_16""",289.584,24.278,313.862,"""gore"""
"""gen_2000_debat…","""SPEAKER_16""",314.27,4.261,318.53098,"""gore"""


In [12]:
def select_samples(data, target_duration=120):
    selected_samples = []

    # Group the data by speaker
    grouped_data = data.group_by('speaker_val')

    for group_key, group in grouped_data:
        # Shuffle the video clips for each speaker
        group = group.sample(fraction=1)

        cumulative_duration = 0
        for row in group.rows(named = True):
            # Add the video clip duration to the cumulative duration
            cumulative_duration += row['duration']

            # Append the selected sample to the result
            selected_samples.append(row)

            # Check if the cumulative duration exceeds the target duration
            if cumulative_duration > target_duration:
                break

    # Convert the list of selected samples to a Polars DataFrame
    selected_samples_df = pl.DataFrame(selected_samples)

    return selected_samples_df

In [13]:
selected_samples_df = select_samples(df)

/var/folders/bv/k0xccxcn6b15hwml87syxhv00000gn/T/ipykernel_64142/54793430.py:7: DeprecationWarning: `group_by` iteration will change to always return group identifiers as tuples. Pass `by` as a list to silence this warning, e.g. `group_by(['speaker_val'])`.
  for group_key, group in grouped_data:


In [14]:
selected_samples_df.write_csv("../data/embedding_data_small.csv")